In [2]:
from glob import glob
# import sys
from dotenv import load_dotenv

load_dotenv()
# sys.path.append("/u/zliu/datastor1/KE-by-CP")
import pandas as pd
# from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import describe
from thefuzz import fuzz

from datasets import load_dataset, load_from_disk

from copy import deepcopy

from dateutil.parser import parse
from dateutil.parser import ParserError

from collections import defaultdict
import string

import re


In [25]:
popular_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/popular.json")
assert len(popular_examples) == 1
popular_examples = popular_examples[0]

random_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/random.json")
assert len(random_examples) == 1
random_examples = random_examples[0]

recent_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/benchmark/recent.json")
assert len(recent_examples) == 1
recent_examples = recent_examples[0]

ripple_edits_examples = random_examples # random_examples # popular_examples + recent_examples + random_examples
len(ripple_edits_examples)

1922

In [26]:
0.15 * len(ripple_edits_examples)

288.3

In [27]:
# random_examples[0]


In [28]:
non_zero_outerloop_count = 0
strong_meta_examples = []
weak_meta_examples = []
for example in ripple_edits_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    # non_zero_outerloop_count += len(outerloop_instances) > 0
    if len(outerloop_instances) > 0 and len(locality_instances) > 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        strong_meta_examples.append(example)
    elif len(locality_instances) == 0:
        
        outerloop_queries = [q for instance in outerloop_instances for q in instance["test_queries"]]
        outerloop_queries = [q for q in outerloop_queries if len(q["answers"]) > 0]
        outerloop_queries = [q for q in outerloop_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in outerloop_queries])
        
        if len(outerloop_queries) == 0:
            continue
        
        weak_meta_examples.append(example)
    else:
        locality_queries = [q for instance in locality_instances for q in instance["test_queries"]]
        locality_queries = [q for q in locality_queries if len(q["answers"]) > 0]
        locality_queries = [q for q in locality_queries if len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0]
        assert all([len(q["prompt"].strip()) > 0 for q in locality_queries])
        
        assert len(locality_queries) > 0
        
        weak_meta_examples.append(example)
        
print("# strong examples", len(strong_meta_examples))
print("# weak examples", len(weak_meta_examples))

# strong examples 1413
# weak examples 503


In [29]:
len(strong_meta_examples) + len(weak_meta_examples)

1916

In [31]:
# 0.15 * 1916

In [43]:
# [[a["value"] for a in q["answers"] if len(a["value"].strip()) > 0] for q in locality_queries]

In [32]:
n_test = 140
n_valid = 140
np.random.shuffle(strong_meta_examples)
np.random.shuffle(weak_meta_examples)

test_examples = strong_meta_examples[:n_test]
valid_examples = strong_meta_examples[n_test:n_test+n_valid]
train_examples = strong_meta_examples[n_test+n_valid:] + weak_meta_examples
np.random.shuffle(train_examples)

In [ ]:
# os.makedirs(f"{vars.DATA_DIR}/ripple_edits/meta_train_random/", exist_ok=True)

In [35]:
io.dump_jsonlines(train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_random/train.jsonl")
io.dump_jsonlines(valid_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_random/valid.jsonl")
io.dump_jsonlines(test_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_random/test.jsonl")

In [36]:
len(train_examples)

1636

In [38]:
recent_popular_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train.jsonl")

In [39]:
random_train = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_random/train.jsonl")

In [ ]:
# io.dump_jsonlines(recent_popular_train + random_train, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/train_w_random.jsonl")

### Check spurious correlation in dataaset (answer verbtaim in edit)

In [3]:
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/test.jsonl")

In [8]:
len(train_examples)

200

In [4]:
non_zero_outerloop_count = 0
meta_examples = []
propagation2verbtaim_count = defaultdict(int)
propagation2count = defaultdict(int)
# strong_meta_examples = []
# weak_meta_examples = []
count_rel = defaultdict(int)
count_prop_types = defaultdict(int)

for example in train_examples:
    outerloop_instances = example["Logical_Generalization"] + example["Compositionality_I"] + example["Compositionality_II"] + example["Subject_Aliasing"]
    # for ins in outerloop_instances:
        # assert len(ins["test_queries"]) == 1
    locality_instances = example["Relation_Specificity"] + example["Forgetfulness"] 
    example["edit"]["text"] = example["edit"]["prompt"]
    meta_examples.append(example["edit"])
    
    outerloop_queries = []
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    outerloop_queries.append(q)
    for outer_q in outerloop_queries:
        count_prop_types["efficacy::"+ outer_q["question_type"]] += 1
        count_rel["efficacy::"+ outer_q["relation"]] += 1
    # assert len(outerloop_queries) > 0
    meta_examples.extend(outerloop_queries)
    
    locality_queries = []
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    q["text"] = q["prompt"] + " " + ans_candidates[0]
                    propagation2count[k] += 1
                    propagation2verbtaim_count[k] += int(ans_candidates[0] in example["edit"]["text"])
                    locality_queries.append(q)
    for loc_q in locality_queries:
        count_prop_types["specificity::"+ loc_q["question_type"]] += 1
        count_rel["specificity::"+ loc_q["relation"]] += 1
    
    meta_examples.extend(locality_queries)
    
    # assert len(locality_queries) > 0
        
# print("# strong examples", len(strong_meta_examples))
# print("# weak examples", len(weak_meta_examples))

In [5]:
propagation2count

defaultdict(int,
            {'Logical_Generalization': 122,
             'Relation_Specificity': 933,
             'Compositionality_I': 995,
             'Compositionality_II': 106,
             'Subject_Aliasing': 291,
             'Forgetfulness': 119})

In [11]:
propagation2verbtaim_ratio = {k: round(propagation2verbtaim_count[k] / propagation2count[k] * 100, 1) for k in propagation2count}

In [12]:
propagation2verbtaim_ratio

{'Logical_Generalization': 42.6,
 'Relation_Specificity': 2.5,
 'Compositionality_I': 3.2,
 'Compositionality_II': 100.0,
 'Subject_Aliasing': 100.0,
 'Forgetfulness': 54.6}

In [16]:
ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/test_w_prefix.hf")

In [17]:
test_data = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/test.jsonl")

In [18]:
len(ds)
len(test_data)

200

In [14]:
ds

Dataset({
    features: ['prompt', 'subject_id', 'relation', 'target_id', 'original_fact', 'context', 'paraphrase', 'object'],
    num_rows: 200
})

In [19]:
new_test_data = []

count = 0
for i in range(len(test_data)):
    original_test = test_data[i]
    augmentation = ds[i-count]
    if original_test["edit"]["prompt"] == "The name of the composer of Klavierstücke  I–IV is Karlheinz Stockhausen.":
        # print("found")
        original_test["edit"]["subject"] = "name of the composer of Klavierst\u00fccke  I\u2013IV"
        original_test["edit"]["object"] = "Karlheinz Stockhausen"
        assert original_test["edit"]["subject"] in original_test["edit"]["prompt"]
        assert original_test["edit"]["object"] in original_test["edit"]["prompt"]
        new_test_data.append(original_test)
        count += 1
        continue
        # print(original_test)
        # print(augmentation)
    else:
        
        assert original_test["edit"]["prompt"] == augmentation["prompt"], f"{i}@@" + original_test["edit"]["prompt"] + "@@" + augmentation["prompt"]
        assert augmentation["subject"] in original_test["edit"]["prompt"]
        assert augmentation["object"] in original_test["edit"]["prompt"]

        original_test["edit"]["subject"] = augmentation["subject"]
        original_test["edit"]["object"] = augmentation["object"]
        new_test_data.append(original_test)
    

KeyError: 'subject'

In [96]:
io.dump_jsonlines(new_test_data, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/test_aug.jsonl")


### Prepare a ripple edits for MEND-original version (before generating paraphrase)

In [3]:
split = "test"
train_examples = io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}.jsonl")
train_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_w_prefix.hf")

# special treatment for test set

In [4]:

new_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    new_example = deepcopy(example)
    
    new_example["edit"]["context"] = augmentation["context"]
    new_example["edit"]["paraphrase"] = augmentation["paraphrase"]
    new_example["edit"]["object"] = augmentation["object"]
    new_examples.append(new_example)
    # if example["edit"]["prompt"] != augmentation["prompt"]:
    #     mismatch_c += 1
    # else:
        
    #     assert example["edit"]["prompt"] == augmentation["prompt"]
    #     assert augmentation["object"] in example["edit"]["prompt"]
    
    #     flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["object"], "paraphrase": augmentation["paraphrase"]})
    

In [ ]:
# if split == "test":
#     io.dump_jsonlines(new_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_mend.jsonl")

In [88]:
flattend_train_examples = []
missing_examples = []
mismatch_c = 0
for i in range(len(train_examples)):
    augmentation = train_ds[i-mismatch_c]
    example = train_examples[i]
    
    if example["edit"]["prompt"] != augmentation["prompt"]:
        mismatch_c += 1
    else:
        
        assert example["edit"]["prompt"] == augmentation["prompt"]
        assert augmentation["object"] in example["edit"]["prompt"], augmentation["object"] + "@@@" + example["edit"]["prompt"]
    
        flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["object"], "paraphrase": augmentation["paraphrase"]})
    
    for k in ["Logical_Generalization", "Compositionality_I", "Compositionality_II", "Subject_Aliasing"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in ".",  q["prompt"]
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0],})
    for k in ["Relation_Specificity", "Forgetfulness"]:
        for instance in example[k]:
            for q in instance["test_queries"]:
                if len(q["answers"]) > 0 and len([a["value"] for a in q["answers"] if len(a["value"].strip() ) > 0 ]) > 0:
                    q["question_type"] = k
                    ans_candidates = [a["value"] for a in q["answers"] if len(a["value"].strip()) > 0]
                    assert len(ans_candidates) > 0
                    assert q["prompt"][-1] not in string.punctuation
                    
                    assert "phrase" in q
                    if q["phrase"] is not None:
                        flattend_train_examples.append({"context": q["prompt"], "completion": ans_candidates[0], "paraphrase": q["phrase"]})
                    else:
                        missing_examples.append({"context": q["prompt"], "completion": ans_candidates[0],})
    

In [74]:
# missing_ds = load_from_disk(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/{split}_w_paraphrase.hf")
# for i in range(len(missing_ds)):
#     augmentation = missing_ds[i]
    
#     flattend_train_examples.append({"context": augmentation["context"], "completion": augmentation["completion"], "paraphrase": augmentation["paraphrase"]})

In [89]:
len(flattend_train_examples)

17096

In [ ]:
# io.dump_jsonlines(flattend_train_examples, f"{vars.DATA_DIR}/ripple_edits/meta_train_recent+popular/{split}_mend.jsonl")

In [78]:
for ex in io.load_jsonlines(f"{vars.DATA_DIR}/ripple_edits/meta_train_recent/valid_mend.jsonl"):
    assert "context" in ex and ex["context"] is not None
    assert "completion" in ex and ex["completion"] is not None
    assert "paraphrase" in ex and ex["paraphrase"] is not None

In [4]:
recoe_aggregation = io.load_json("/u/zliu/datastor1/ReCoE/data/aggregation/counterfactual_datapoints_verified_atomic.json")

In [52]:
converted_recoe_aggregation = [
    {
        "text": " ".join([f["fact"] for f in x["direct_counterfactual_fact"]]),
        "question": x["question"],
        "answers": x["counterfactual_answer"]
     }
    for x in recoe_aggregation
]


In [11]:
converted_recoe_aggregation[-1]
len(converted_recoe_aggregation)

508

### Integrate RoCE data to ripple edit trainings

In [12]:
recoe_comparative = io.load_json("/u/zliu/datastor1/ReCoE/data/comparative/counterfactual_datapoints_verified_atomic.json")

In [51]:
# [len(x["facts_per_choice"]) for x in recoe_comparative]
# recoe_comparative[0]

In [ ]:
converted_recoe_comparative = []
for x in recoe_comparative:
    assert "choice_1_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_2_counterfactuals" in x["counterfactuals_per_choice"]
    assert "choice_3_counterfactuals" not in x["counterfactuals_per_choice"]
    
    counter_fact1_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_1_counterfactuals"]]
    counter_fact2_text = [f["fact"] for f in x["counterfactuals_per_choice"]["choice_2_counterfactuals"]]
    texts = [counter_fact1_text, counter_fact2_text]
    np.random.shuffle(texts)
    counter_fact1_text_after_rand, counter_fact2_text_after_rand = texts
    
    
    converted_recoe_comparative.append(
        {
            "text": " ".join(counter_fact1_text_after_rand + counter_fact2_text_after_rand),
            "question": x["question"],
            "answers": x["counterfactual_answer"]
        }
    )

In [48]:
np.random.shuffle(texts)

In [13]:
# [len(x["facts_per_choice"]) for x in recoe_comparative]
recoe_comparative = io.load_json("/u/zliu/datastor1/ReCoE/data/counting/counterfactual_datapoints_verified_atomic.json")

In [15]:
recoe_comparative[0]

{'question': 'How many dwarf planets are recognized by the International Astronomical Union (IAU)?',
 'answer': '5',
 'domain': 'astronomy',
 'links': ['https://en.wikipedia.org/wiki/List_of_possible_dwarf_planets'],
 'counterfactual_facts': [{'original_output': 'Pluto - Discovered in 1930, but no longer considered a dwarf planet by the IAU since 2015.',
   'sentence': 'Pluto was discovered in 1930, but has not been considered a dwarf planet by the IAU since 2015.',
   'atomic_facts': ['Pluto was discovered in 1930',
    'Pluto has not been considered a dwarf planet by the IAU since 2015',
    'Pluto not being considered a dwarf planet by the IAU has been the case since 2015'],
   'atomic_triples': ['(Pluto; was discovered in; 1930)',
    '(Pluto; has not been considered a dwarf planet by; the IAU)',
    '(Pluto not being considered a dwarf planet by the IAU; has been the case since; 2015)']}],
 'counterfactual_answer': '4',
 'facts': [{'original_output': 'Pluto - Discovered in 1930, c